In [1]:
import numpy as np
import os

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from keras.layers import Input, Dense, Flatten, Convolution2D, BatchNormalization, Activation, Add
from keras.models import Model
from keras.utils import to_categorical
from keras import backend as K
from pommerman.constants import BOARD_SIZE
from tqdm import tqdm_notebook as tqdm
from sklearn.utils.class_weight import compute_class_weight

/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Import error! You will not be able to render --> Cannot connect to "None"


In [2]:
batch_size = 2048
epochs = 150
early_stopping = 5
action_space = 6

log_path = './supervised_learning/logs/5cnn/'
model_path = './supervised_learning/model/5cnn/model.h4'

train_data_path    = './dataset/'
train_data_labels  = os.path.join(train_data_path, 'labels.npy')
train_data_reward  = os.path.join(train_data_path, 'reward.npy')
train_data_obs_map = os.path.join(train_data_path, 'obs_map.npy')

if not os.path.isdir(train_data_path):
    os.makedirs(train_data_path)

In [17]:
class Trainer:
    def __init__(self, actions, save_path, log_path, save_best_only=True, seed=0):
        K.clear_session()
        self.log_path = log_path
        self.save_path = save_path
        self.actions = actions
        self.save_best_only = save_best_only
        
        # Create model
        self.model = self.create_model(actions)
        # Load model if exists
        if not os.path.isdir(os.path.dirname(save_path)):
            os.makedirs(os.path.dirname(save_path))            
        if os.path.isfile(self.save_path):
            try:
                print("Trying to load model")
                self.model.load_weights(self.save_path)
                print("Model was loaded successful")
            except:
                print("Model load failed")
        
    def create_model(self, actions, input_shape=(11, 11, 18,)):
        inp = Input(input_shape)
        # 5 Convs -> receptive field 11
        x = Convolution2D(256, 3, padding='valid')(inp)
        x = BatchNormalization()(x)
        x = Activation('elu')(x)
        x = Convolution2D(256, 3, padding='valid')(x)
        x = BatchNormalization()(x)
        x = Activation('elu')(x)
        x = Convolution2D(256, 3, padding='valid')(x)
        x = BatchNormalization()(x)
        x = Activation('elu')(x)
        x = Convolution2D(256, 3, padding='valid')(x)
        x = BatchNormalization()(x)
        x = Activation('elu')(x)
        x = Convolution2D(256, 3, padding='valid')(x)
        x = BatchNormalization()(x)
        x = Activation('elu')(x)
        x = Flatten()(x)
        # Output
        probs  = Dense(actions, activation='softmax', name='actions')(x)
        reward = Dense(1, activation='tanh', name='reward')(x)
        
        model = Model(inputs = inp, outputs=[probs, reward])
        model.compile(optimizer='adam', loss=['categorical_crossentropy', 'mae'], metrics=['accuracy'])
        return model

    def train(self, obs, actions, rewards, batch_size=16384, epochs=100,
              early_stopping = 10, class_weight=None, initial_epoch=0):
        
        early_stopping = EarlyStopping(monitor='loss', patience=early_stopping)
        checkpoint     = ModelCheckpoint(self.save_path, monitor='loss', save_best_only=self.save_best_only)
        reduce_lr      = ReduceLROnPlateau(monitor='loss', patience=2, factor=0.1)
        logger         = CSVLogger(self.log_path + 'log.csv', append=True)
        tensorboard    = TensorBoard(self.log_path, batch_size=batch_size)
        
        history = self.model.fit(x=obs, y=[actions, rewards], batch_size=batch_size, epochs=epochs, verbose=1,
                       callbacks=[early_stopping, checkpoint, reduce_lr, logger, tensorboard],
                       validation_split=0.15, shuffle=True, class_weight=class_weight, initial_epoch=initial_epoch)

In [4]:
labels       = np.load(train_data_labels)
observations = np.load(train_data_obs_map)
rewards      = np.load(train_data_reward)

In [5]:
labels = to_categorical(labels, num_classes=action_space)

In [6]:
labels.shape, observations.shape, rewards.shape

((553065, 6), (553065, 11, 11, 18), (553065,))

In [7]:
np.sum(labels, axis=0) / np.sum(labels)

array([0.15941164, 0.20397964, 0.190945  , 0.20009764, 0.20339382,
       0.04217226], dtype=float32)

In [8]:
class_weights = compute_class_weight('balanced', np.unique(np.argmax(labels, axis=1)), np.argmax(labels, axis=1))
class_weights

array([1.04551126, 0.81707501, 0.87285166, 0.83292671, 0.81942839,
       3.9520451 ])

In [18]:
trainer = Trainer(action_space, model_path, log_path)

In [19]:
trainer.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 11, 11, 18)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 9, 9, 256)    41728       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 9, 9, 256)    1024        conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 9, 9, 256)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
trainer.train(observations, labels, rewards, batch_size=batch_size, 
              epochs=epochs, early_stopping=early_stopping, class_weight=[class_weights, 1])

Train on 470105 samples, validate on 82960 samples
Epoch 1/150
470105/470105 [==============================] - 40s 84us/step - loss: 2.2331 - actions_loss: 1.6744 - reward_loss: 0.5586 - actions_acc: 0.2571 - reward_acc: 0.7176 - val_loss: 2.1872 - val_actions_loss: 1.6816 - val_reward_loss: 0.5056 - val_actions_acc: 0.2487 - val_reward_acc: 0.7480
Epoch 2/150
470105/470105 [==============================] - 36s 77us/step - loss: 1.9856 - actions_loss: 1.5082 - reward_loss: 0.4775 - actions_acc: 0.3359 - reward_acc: 0.7593 - val_loss: 1.9725 - val_actions_loss: 1.4353 - val_reward_loss: 0.5372 - val_actions_acc: 0.3725 - val_reward_acc: 0.7260
Epoch 3/150
470105/470105 [==============================] - 36s 77us/step - loss: 1.7433 - actions_loss: 1.2773 - reward_loss: 0.4660 - actions_acc: 0.4303 - reward_acc: 0.7648 - val_loss: 1.7583 - val_actions_loss: 1.2539 - val_reward_loss: 0.5044 - val_actions_acc: 0.4486 - val_reward_acc: 0.7478
Epoch 4/150
470105/470105 [===================

Epoch 28/150
470105/470105 [==============================] - 57s 120us/step - loss: 0.7603 - actions_loss: 0.6338 - reward_loss: 0.1265 - actions_acc: 0.7100 - reward_acc: 0.9355 - val_loss: 1.7005 - val_actions_loss: 1.0792 - val_reward_loss: 0.6212 - val_actions_acc: 0.5775 - val_reward_acc: 0.6806
Epoch 29/150
470105/470105 [==============================] - 56s 120us/step - loss: 0.7428 - actions_loss: 0.6224 - reward_loss: 0.1204 - actions_acc: 0.7153 - reward_acc: 0.9386 - val_loss: 1.6741 - val_actions_loss: 1.0614 - val_reward_loss: 0.6127 - val_actions_acc: 0.5924 - val_reward_acc: 0.6843
Epoch 30/150
470105/470105 [==============================] - 59s 125us/step - loss: 0.7296 - actions_loss: 0.6129 - reward_loss: 0.1167 - actions_acc: 0.7197 - reward_acc: 0.9405 - val_loss: 1.7086 - val_actions_loss: 1.0437 - val_reward_loss: 0.6649 - val_actions_acc: 0.5861 - val_reward_acc: 0.6569
Epoch 31/150
470105/470105 [==============================] - 55s 118us/step - loss: 0.7129

470105/470105 [==============================] - 58s 123us/step - loss: 0.3997 - actions_loss: 0.3507 - reward_loss: 0.0491 - actions_acc: 0.8486 - reward_acc: 0.9751 - val_loss: 2.7031 - val_actions_loss: 1.9717 - val_reward_loss: 0.7315 - val_actions_acc: 0.5725 - val_reward_acc: 0.6249
Epoch 56/150
470105/470105 [==============================] - 60s 127us/step - loss: 0.3916 - actions_loss: 0.3435 - reward_loss: 0.0481 - actions_acc: 0.8516 - reward_acc: 0.9756 - val_loss: 2.4500 - val_actions_loss: 1.8398 - val_reward_loss: 0.6102 - val_actions_acc: 0.5743 - val_reward_acc: 0.6880
Epoch 57/150
470105/470105 [==============================] - 59s 124us/step - loss: 0.3823 - actions_loss: 0.3345 - reward_loss: 0.0478 - actions_acc: 0.8556 - reward_acc: 0.9757 - val_loss: 2.4585 - val_actions_loss: 1.8287 - val_reward_loss: 0.6298 - val_actions_acc: 0.5683 - val_reward_acc: 0.6773
Epoch 58/150
470105/470105 [==============================] - 59s 125us/step - loss: 0.3726 - actions_lo

In [ ]:
print("finish")

In [ ]:
trainer.model.save(model_path)